In [278]:
import pandas as pd

In [279]:
def input_loading_values():
    loading_values = {}
    loading_order = [
        "cladding",
        "services",
        "PV loading",
        "maintenance",
        "uniform snow loading",
        "snow drift valley",
        "snow drift parapet",
        "wind down",
        "snow drift length"
    ]

    for value_name in loading_order:
        value = float(input(f"Enter loading value for {value_name}: "))
        loading_values[value_name] = value
        
    loading_values['dead loading'] = loading_values['cladding'] + loading_values['services']
    loading_values['existing loading'] =  loading_values['dead loading'] + loading_values['maintenance']
    loading_values['proposed loading'] = loading_values['dead loading'] + loading_values['PV loading']

    return loading_values

In [280]:
def load_increase_typical_area(loading_values):
    sum_1 = loading_values["existing loading"]
    sum_2 = loading_values["proposed loading"] + loading_values["uniform snow loading"]

    percentage_increase = ((sum_2 - sum_1) / sum_1) * 100

    return percentage_increase

In [281]:
def load_increase_snow_valley(loading_values):
    sum_1 = loading_values["existing loading"]
    sum_2 = loading_values["proposed loading"] + loading_values["snow drift valley"]

    percentage_increase = ((sum_2 - sum_1) / sum_1) * 100

    return percentage_increase

In [282]:
def load_increase_snow_parapet(loading_values):
    sum_1 = loading_values["existing loading"]
    sum_2 = loading_values["proposed loading"] + loading_values["snow drift parapet"]

    percentage_increase = ((sum_2 - sum_1) / sum_1) * 100

    return percentage_increase

In [283]:
def add_new_loading(dictionary):
    load_name = input('Enter a description/type of loading: ')
    load_value = float(input('Enter the load value in kPa: '))
    
    dictionary[load_name] = load_value

In [284]:
def existing_LC_BS(load_dict):
    # Creates an empty dictionary to store the exisitng conditions load combination with 
    # Britist Standard factors of safety (exist_lc_bs)
    exist_lc_bs = {}
    
    # Creating new key value pairs for dictionary 'existing_lc' and calculating load combinations values
    exist_lc_bs['exist_lc_1'] = 1.4 * load_dict['dead loading'] + 1.6 * (max(load_dict['maintenance'],load_dict['uniform snow loading']))
    exist_lc_bs['exist_lc_2'] = 1.4 * load_dict['dead loading'] + 1.4 * load_dict['wind down']
    exist_lc_bs['exist_lc_3'] = 1.2 * load_dict['dead loading'] + 1.2 * load_dict['wind down'] + 1.2 * (max(load_dict['maintenance'],load_dict['uniform snow loading']))
    exist_lc_bs['exist_lc_4'] = 1.4 * load_dict['dead loading'] + 1.05 * load_dict['snow drift valley']
    exist_lc_bs['exist_lc_5'] = 1.4 * load_dict['dead loading'] + 1.05 * load_dict['snow drift parapet']
    exist_lc_bs['exist_lc_6'] = 1 * load_dict['dead loading'] + 1 * (max(load_dict['maintenance'],load_dict['uniform snow loading']))
    exist_lc_bs['exist_lc_7'] = 1 * load_dict['dead loading'] + 1 * load_dict['uniform snow loading']
    exist_lc_bs['exist_lc_8'] = 1 * load_dict['dead loading'] + 0.8 * load_dict['wind down'] + 0.8 * (max(load_dict['maintenance'],load_dict['uniform snow loading']))
    
    # Finding max values for certain load combinations
    exist_lc_bs['exist_max_1'] = (max(exist_lc_bs['exist_lc_1'],exist_lc_bs['exist_lc_2'],exist_lc_bs['exist_lc_3'])) 
    exist_lc_bs['exist_max_2'] = exist_lc_bs['exist_lc_4']
    exist_lc_bs['exist_max_3'] = exist_lc_bs['exist_lc_5']
    exist_lc_bs['exist_max_4'] = ((max(exist_lc_bs['exist_lc_6'],exist_lc_bs['exist_lc_7'],exist_lc_bs['exist_lc_8'])))
                           
    return exist_lc_bs

In [285]:
def proposed_LC_BS(load_dict):
    # Creates an empty dictionary to store the proposed conditions load combination with 
    # Britist Standard factors of safety (prop_lc_bs)
    prop_lc_bs = {}
    
    # Creating new key value pairs for dictionary 'existing_lc' and calculating load combinations values
    prop_lc_bs['prop_lc_1'] = 1.4 * load_dict['dead loading'] + 1.6 * (max(load_dict['maintenance'],load_dict['uniform snow loading']))
    prop_lc_bs['prop_lc_2'] = 1.4 * load_dict['dead loading'] + 1.4 * load_dict['wind down']
    prop_lc_bs['prop_lc_3'] = 1.2 * load_dict['dead loading'] + 1.2 * load_dict['wind down'] + 1.2 * (max(load_dict['maintenance'],load_dict['uniform snow loading']))
    prop_lc_bs['prop_lc_4'] = 1.4 * load_dict['dead loading'] + 1.05 * load_dict['snow drift valley']
    prop_lc_bs['prop_lc_5'] = 1.4 * load_dict['dead loading'] + 1.05 * load_dict['snow drift parapet']
    prop_lc_bs['prop_lc_6'] = 1 * load_dict['dead loading'] + 1 * (max(load_dict['maintenance'],load_dict['uniform snow loading']))
    prop_lc_bs['prop_lc_7'] = 1 * load_dict['dead loading'] + 1 * load_dict['uniform snow loading']
    prop_lc_bs['prop_lc_8'] = 1 * load_dict['dead loading'] + 0.8 * load_dict['wind down'] + 0.8 * (max(load_dict['maintenance'],load_dict['uniform snow loading']))
    
    # Finding max values for certain groups of load combinations
    prop_lc_bs['prop_max_1'] = (max(prop_lc_bs['prop_lc_1'],prop_lc_bs['prop_lc_2'],prop_lc_bs['prop_lc_3'])) 
    prop_lc_bs['prop_max_2'] = prop_lc_bs['prop_lc_4']
    prop_lc_bs['prop_max_3'] = prop_lc_bs['prop_lc_5']
    prop_lc_bs['prop_max_4'] = (max(prop_lc_bs['prop_lc_6'],prop_lc_bs['prop_lc_7'],prop_lc_bs['prop_lc_8'])) 
    
    return prop_lc_bs

In [286]:
def purlin_arrangement():
    # Empty dictionaries to store calculation values
    distance_to_next_purlin = {}
    distance_from_ridge = {}
    load_width = {}
    purlin_capacities = {}
    keys = ['ULS Capacity', 'SLS Capacity']
    purlin_ULS_capacity = pd.Series(dtype=float)
    purlin_SLS_capacity = pd.Series(dtype=float)
    
    # Ask user to input the roof structures measurements from the survey 
    purlin_no = int(input('Please enter the number of purlins: '))
    span = float(input('Please enter the max purlin span: '))
    purlin_type = str(input('Please enter purlin type: '))
        
    # Ask user to input the spacing between each purlin starting from the lower edge
    for i in range(purlin_no):
        distance_to_next_purlin[i] = float(input(f'Input the spacing between purlin {i} to purlin {i+1}: '))
    
    # Ask user to input the ULS and SLS capacities
    for key in keys:
        purlin_capacities[key] = int(input(f'Enter {key} of the purlin: '))
    
    # Creates a series of the ULS capacities the same length as the number of purlins to compare 
    # against calculated values in the final table
    for i in range(1, purlin_no):
        value = purlin_capacities['ULS Capacity']
        purlin_ULS_capacity = purlin_ULS_capacity.append(pd.Series([value], index=[i]))
    purlin_ULS_capacity.name = (purlin_type + ' ULS Capacity (kN)')
    
    # Creates a series of the SLS capacities the same length as the number of purlins to compare 
    # against calculated values in the final table
    for i in range(1, purlin_no):
        value = purlin_capacities['SLS Capacity']
        purlin_SLS_capacity = purlin_SLS_capacity.append(pd.Series([value], index=[i]))
    purlin_SLS_capacity.name = (purlin_type + ' SLS Capacity (kN)')
    
    # Calculates the distance each purlin is from the ridge
    distance_from_ridge[0] = sum(distance_to_next_purlin.values())
    for i in range(1, purlin_no):
        distance_from_ridge[i] = distance_from_ridge[i-1] - distance_to_next_purlin[i-1]
    
    # Calculates the load width for each purlin based on their spacing between adjacent purlins
    load_width[1] = distance_to_next_purlin[0] + distance_to_next_purlin[1]/2
    for i in range(2, purlin_no):
        load_width[i] = (distance_to_next_purlin[i-1] + distance_to_next_purlin[i]) /2
    
    return purlin_no, span, distance_to_next_purlin, distance_from_ridge, load_width, purlin_capacities, purlin_type, purlin_ULS_capacity, purlin_SLS_capacity             

In [287]:
def purlin_load_valley(load_dict,purlin_spacing):

    purlin_no = purlin_spacing[0]
    span = purlin_spacing[1]
    distance_to_next_purlin = purlin_spacing[2]
    distance_from_ridge = purlin_spacing[3]
    load_width = purlin_spacing[4]
    purlin_capacities = purlin_spacing[5]
    purlin_type = purlin_spacing[6]
    purlin_ULS_capacity = purlin_spacing[7]
    purlin_SLS_capacity = purlin_spacing[8]
    
    # Empty dictionaries to store calculation values
    snow_drift_valley = pd.Series(dtype=float)
    ULS_load_valley = pd.Series(dtype=float)    
    total_ULS_load_valley = pd.Series(dtype=float)
    ULS_check = pd.Series(dtype=float)
        
    # Calculates snow drift load based on distance from the ridge
    for i in range(1, purlin_no):
        value = (load_dict['snow drift valley'] * distance_from_ridge[i]) / distance_from_ridge[0]
        snow_drift_valley = snow_drift_valley.append(pd.Series([value], index=[i]))         
    snow_drift_valley.name = 'Snow drift loading on purlin (kPa)'
    
    # Calculates ULS load for each purlin from the snow drift for that purlin and the dead loading
    for i in range(1, purlin_no):
        value = 1.4 * load_dict['dead loading'] + 1.05 * snow_drift_valley[i] 
        ULS_load_valley = ULS_load_valley.append(pd.Series([value], index=[i]))
    ULS_load_valley.name = 'ULS area load on valley purlins (kPa)'
    
    # Calculates the total ULS load on each purlin
    for i in range(1, purlin_no):
        value = span * ULS_load_valley[i] * load_width[i]
        total_ULS_load_valley = total_ULS_load_valley.append(pd.Series([value], index=[i]))  
    total_ULS_load_valley.name = 'Total ULS load on valley purlins (kN)'
    
    # Checks the calcualted loading against the ULS capacity to be less than the design load
    for i in range(1, purlin_no):
        if total_ULS_load_valley[i] <= purlin_capacities['ULS Capacity']:
            value = 'OK'
        else:
            value = 'Not OK'
        ULS_check = ULS_check.append(pd.Series([value], index=[i]))
    ULS_check.name = 'Capacity'
    
    # Combines results of each series and brings them into 1 data frame
    df_purlin_load_valley = pd.concat([snow_drift_valley, ULS_load_valley, total_ULS_load_valley, purlin_ULS_capacity, ULS_check], axis=1)
    
    return df_purlin_load_valley

In [288]:
def purlin_load_parapet(load_dict, purlin_spacing):

    purlin_no = purlin_spacing[0]
    span = purlin_spacing[1]
    distance_to_next_purlin = purlin_spacing[2]
    distance_from_ridge = purlin_spacing[3]
    load_width = purlin_spacing[4]
    purlin_capacities = purlin_spacing[5]
    purlin_type = purlin_spacing[6]
    purlin_ULS_capacity = purlin_spacing[7]
    purlin_SLS_capacity = purlin_spacing[8]
    
    # Empty dictionaries to store calculation values
    snow_drift_parapet = pd.Series(dtype=float)
    ULS_load_parapt = pd.Series(dtype=float)   
    total_ULS_load_parapt = pd.Series(dtype=float)
    dist_from_end_drift = {}
    ULS_check = pd.Series(dtype=float)
    
    # Works out the distance at which the parapet snow drift will reduced linearly to 0
    summ = 0    
    while summ <= load_dict['snow drift length']:  
        for i in range(purlin_no):
            summ += distance_to_next_purlin[i]
            drift_end = i
    
    # Works out distance of each purlin from the end of the snow drift
    dist_from_end_drift[1] = load_dict['snow drift length'] - distance_to_next_purlin[0]
    for i in range(2, drift_end):
        dist_from_end_drift[i] = dist_from_end_drift[i-1] - distance_to_next_purlin[i-1]
    
    # Calculates the snow drift parapet loading on each purlin reducing linearly to 0 from the parapet
    for i in range(1, drift_end):
        value = load_dict['snow drift parapet'] * dist_from_end_drift[i] / load_dict['snow drift length']
        snow_drift_parapet = snow_drift_parapet.append(pd.Series([value], index=[i]))      
    snow_drift_parapet.name = 'Snow drift loading from parapet (kPa)'
    
    # Calculates ULS area load for each purlin from the snow drift for that purlin and the dead loading
    for i in range(1, drift_end):
        value = 1.4 * load_dict['dead loading'] + 1.05 * snow_drift_parapet[i]
        ULS_load_parapt = ULS_load_parapt.append(pd.Series([value], index=[i])) 
    ULS_load_parapt.name = 'ULS area load on parapet purlins (kPa)'  

    # Calculates the total ULS load on each purlin from the parapet snow loading
    for i in range(1, drift_end):
        value = span * ULS_load_parapt[i] * load_width[i]
        total_ULS_load_parapt = total_ULS_load_parapt.append(pd.Series([value], index=[i]))
    total_ULS_load_parpt.name = 'Total ULS load on parapet purlins (kN)'
    
    # Checks the calcualted loading against the ULS capacity to be less than the design load
    for i in range(1, drift_end):
        if total_ULS_load_parpt[i] <= purlin_capacities['ULS Capacity']:
            value = 'OK'
        else:
            value = 'Not OK'
        ULS_check = ULS_check.append(pd.Series([value], index=[i]))
    ULS_check.name = 'Capacity'
    
    # Combines results of each series and brings them into 1 data frame
    df_purlin_load_parapet = pd.concat([snow_drift_parapet, ULS_load_parapt, total_ULS_load_parpt, ULS-capacity, ULS_check], axis=1)   
           
    return df_purlin_parapet
          

In [289]:
def purlin_load_existing(load_comb, purlin_spacing):
    
    purlin_no = purlin_spacing[0]
    span = purlin_spacing[1]
    distance_to_next_purlin = purlin_spacing[2]
    load_width = purlin_spacing[4]
    purlin_capacities = purlin_spacing[5]
    purlin_type = purlin_spacing[6]
    purlin_ULS_capacity = purlin_spacing[7]
    purlin_SLS_capacity = purlin_spacing[8]
    
    # Empty dictionaries to store calculation values for the existing loading on purlins
    exist_ULS_load = pd.Series(dtype=float)
    exist_SLS_load = pd.Series(dtype=float)
    ULS_check = pd.Series(dtype=float)
    SLS_check = pd.Series(dtype=float)
    
    # Calculates the total existing ULS load on each purlin
    for i in range(1, purlin_no):
        value = span * load_comb['exist_max_1'] * load_width[i]
        exist_ULS_load = exist_ULS_load.append(pd.Series([value], index=[i])) 
    exist_ULS_load.name = 'Total existing ULS load on purlins (kN)'
    
    # Calculates the total existing SLS load on each purlin
    for i in range(1, purlin_no):
        value = span * load_comb['exist_max_4'] * load_width[i]
        exist_SLS_load = exist_SLS_load.append(pd.Series([value], index=[i])) 
    exist_SLS_load.name = 'Total existing SLS load on purlins (kN)'
    
    # Checks the calcualted loading against the ULS capacity to be less than the design load
    for i in range(1, drift_end):
        if exist_ULS_load[i] <= purlin_capacities['ULS Capacity']:
            value = 'OK'
        else:
            value = 'Not OK'
        ULS_check = ULS_check.append(pd.Series([value], index=[i]))
    ULS_check.name = 'Capacity'
    
    # Checks the calcualted loading against the SLS capacity to be less than the design load
    for i in range(1, drift_end):
        if exist_SLS_load[i] <= purlin_capacities['ULS Capacity']:
            value = 'OK'
        else:
            value = 'Not OK'
        ULS_check = ULS_check.append(pd.Series([value], index=[i]))
    ULS_check.name = 'Capacity'
    
    # Combines results of each series and brings them into 1 data frame
    df_pulin_load_existing = pd.concat([exist_ULS_load, purlin_ULS_capacity, ULS_check, exist_SLS_load, purlin_SLS_capacity, SLS_check])
    
    return df_purlin_load_existing

In [290]:
def purlin_load_proposed(load_comb, purlin_spacing):
    
    purlin_no = purlin_spacing[0]
    span = purlin_spacing[1]
    distance_to_next_purlin = purlin_spacing[2]
    load_width = purlin_spacing[4]
    purlin_capacities = purlin_spacing[5]
    purlin_type = purlin_spacing[6]
    purlin_ULS_capacity = purlin_spacing[7]
    purlin_SLS_capacity = purlin_spacing[8]
    
    # Empty dictionaries to store calculation values for the proposed loading on the purlins
    prop_ULS_load = pd.Series(dtype=float)
    prop_SLS_load = pd.Series(dtype=float)
    ULS_check = pd.Series(dtype=float)
    SLS_check = pd.Series(dtype=float)
    
    # Calculates the total proposed ULS load on each purlin
    for i in range(1, purlin_no):
        value = span * load_comb['prop_max_1'] * load_width[i]
        prop_ULS_load = prop_ULS_load.append(pd.Series([value], index=[i])) 
    prop_ULS_load.name = 'Total proposed ULS load on purlins (kN)'
    
    # Calculates the total proposed SLS load on each purlin
    for i in range(1, purlin_no):
        value = span * load_comb['prop_max_4'] * load_width[i]
        prop_SLS_load = prop_SLS_load.append(pd.Series([value], index=[i]))
    prop_SLS_load.name = 'Total proposed SLS load on purlins (kN)'  
    
     # Checks the calcualted loading against the ULS capacity to be less than the design load
    for i in range(1, drift_end):
        if exist_ULS_load[i] <= purlin_capacities['ULS Capacity']:
            value = 'OK'
        else:
            value = 'Not OK'
        ULS_check = ULS_check.append(pd.Series([value], index=[i]))
    ULS_check.name = 'Capacity'
    
    # Checks the calcualted loading against the SLS capacity to be less than the design load
    for i in range(1, drift_end):
        if exist_SLS_load[i] <= purlin_capacities['ULS Capacity']:
            value = 'OK'
        else:
            value = 'Not OK'
        ULS_check = ULS_check.append(pd.Series([value], index=[i]))
    ULS_check.name = 'Capacity'
    
     # Combines results of each series and brings them into 1 data frame
    df_pulin_load_proposed = pd.concat([prop_ULS_load, purlin_ULS_capacity, ULS_check, prop_SLS_load, purlin_SLS_capacity, SLS_check])
    
    return df_pulin_load_proposed

In [292]:
loading_values = input_loading_values()
purlin_survey = purlin_arrangement()
proposed_check = purlin_load_proposed(loading_values, purlin_survey)
proposed_check


Enter loading value for cladding: 0.15
Enter loading value for services: 0.25
Enter loading value for PV loading: 0.15
Enter loading value for maintenance: 0.6
Enter loading value for uniform snow loading: 0.3
Enter loading value for snow drift valley: 1.2
Enter loading value for snow drift parapet: 1.2
Enter loading value for wind down: 0.3
Enter loading value for snow drift length: 6
Please enter the number of purlins: 4
Please enter the max purlin span: 6
Please enter purlin type: metsec
Input the spacing between purlin 0 to purlin 1: 2
Input the spacing between purlin 1 to purlin 2: 2
Input the spacing between purlin 2 to purlin 3: 2
Input the spacing between purlin 3 to purlin 4: 2
Enter ULS Capacity of the purlin: 2
Enter SLS Capacity of the purlin: 2


C:\Users\User\AppData\Local\Temp\ipykernel_7796\4245985776.py:28: FutureWarning: The series.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  purlin_ULS_capacity = purlin_ULS_capacity.append(pd.Series([value], index=[i]))
C:\Users\User\AppData\Local\Temp\ipykernel_7796\4245985776.py:28: FutureWarning: The series.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  purlin_ULS_capacity = purlin_ULS_capacity.append(pd.Series([value], index=[i]))
C:\Users\User\AppData\Local\Temp\ipykernel_7796\4245985776.py:28: FutureWarning: The series.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  purlin_ULS_capacity = purlin_ULS_capacity.append(pd.Series([value], index=[i]))
C:\Users\User\AppData\Local\Temp\ipykernel_7796\4245985776.py:35: FutureWarning: The series.append method is deprecated and will be removed from pandas in

KeyError: 'prop_max_1'